# Bedrock Knowledge Base Retrieval and Generation with SageMaker Inference and Reranking

![Reranking](./reranking.png)

## 1: Import Required Functions

In [1]:
# Import necessary functions from advanced_rag_utils
from advanced_rag_utils import (
    load_variables,
    setup_bedrock_client,
    format_llama3_prompt,
    search_knowledge_base_with_reranking,
    enhanced_generate_sagemaker_response,
    compare_reranking
)

# For the boto3 import
import boto3

## 2: Load Configuration Variables

In [2]:
# Load the configuration variables
variables = load_variables("../variables.json")
variables

{'accountNumber': '977018081517',
 'regionName': 'us-west-2',
 'collectionArn': 'arn:aws:aoss:us-west-2:977018081517:collection/x6j41w4wldt22s0mc4oa',
 'collectionId': 'x6j41w4wldt22s0mc4oa',
 'vectorIndexName': 'ws-index-',
 'bedrockExecutionRoleArn': 'arn:aws:iam::977018081517:role/advanced-rag-workshop-bedrock_execution_role-us-west-2',
 's3Bucket': '977018081517-us-west-2-advanced-rag-workshop',
 'kbFixedChunk': 'WFJRA66ARJ',
 'kbSemanticChunk': 'XRQUIKPQAB',
 'kbHierarchicalChunk': 'DBCDV6VOVB',
 'kbCustomChunk': 'SVUWGDQTQZ',
 'sagemakerLLMEndpoint': 'endpoint-llama-3-2-3b-instruct-2025-05-01-17-04-39',
 'guardrail_id': 'vcd1cw90tu5w',
 'guardrail_version': '1'}

## 3: Define Configuration Details

In [3]:
# Knowledge Base Selection  
kb_id = variables["kbFixedChunk"]  # Options: "kbFixedChunk", "kbHierarchicalChunk", "kbSemanticChunk"

# SageMaker endpoint
sagemaker_endpoint = variables['sagemakerLLMEndpoint']

# Retrieval-Augmented Generation (RAG) Configuration  
number_of_results = 10  # Number of relevant documents to retrieve  
generation_configuration = {
    "temperature": 0,  # Lower temperature for more deterministic responses  
    "top_k": 10,  # Consider top 10 tokens at each generation step  
    "max_new_tokens": 5000,  # Maximum number of tokens to generate  
    "stop": "<|eot_id|>"  # Stop sequence to end the response generation  
}

# Define ARN for the reranking model
rerank_model_arn = "arn:aws:bedrock:us-west-2::foundation-model/cohere.rerank-v3-5:0"

# Default user query
query = "what was the % increase in sales?"

## 4: Initialize Bedrock Client

In [4]:
# Initialize the Bedrock agent runtime client
bedrock_client = setup_bedrock_client(variables["regionName"])

## 5: Compare Search Results With & Without Reranking

In [5]:
# Define a new query for comparison
query = "Compare the results between 2022 and 2023"

# Search without reranking
print("WITHOUT RERANKING:")
context_without_reranking, details_without_reranking = search_knowledge_base_with_reranking(
    query=query,
    knowledge_base_id=kb_id,
    bedrock_client=bedrock_client,
    num_results=number_of_results,
    use_reranking=False,
    region_name=variables["regionName"]
)

WITHOUT RERANKING:

TOP 3 DOCUMENTS WITHOUT RERANKING:
Position 1 (Score: 0):
This was partially offset by a reduction in excess tax benefits from stock-based compensation and a decrease in the tax impact of foreign earnings and losses driven by a decline in the favorable effects of corporate restructuring transactions. The foreign income deduction benefit recognized in 2022 ...

Position 2 (Score: 0):
North America sales increased 12% in 2023, compared to the prior year. The sales growth primarily reflects increased unit sales, primarily by third-party sellers, advertising sales, and subscription services. Increased unit sales were driven largely by our continued focus on price, selection, and co...

Position 3 (Score: 0):
This regime is referred to as the Foreign-Derived Intangible Income deduction and is dependent on the amount of our U.S. taxable income.     We recorded a provision for income taxes in 2023 as compared to an income tax benefit in 2022 primarily due to an increase in

In [6]:
# Search with reranking
print("\nWITH RERANKING:")
context_with_reranking, details_with_reranking = search_knowledge_base_with_reranking(
    query=query,
    knowledge_base_id=kb_id,
    bedrock_client=bedrock_client,
    num_results=number_of_results,
    use_reranking=True,
    rerank_model_arn=rerank_model_arn,
    region_name=variables["regionName"]
)


WITH RERANKING:

TOP 3 DOCUMENTS WITHOUT RERANKING:
Position 1 (Score: 0):
This was partially offset by a reduction in excess tax benefits from stock-based compensation and a decrease in the tax impact of foreign earnings and losses driven by a decline in the favorable effects of corporate restructuring transactions. The foreign income deduction benefit recognized in 2022 ...

Position 2 (Score: 0):
North America sales increased 12% in 2023, compared to the prior year. The sales growth primarily reflects increased unit sales, primarily by third-party sellers, advertising sales, and subscription services. Increased unit sales were driven largely by our continued focus on price, selection, and co...

Position 3 (Score: 0):
This regime is referred to as the Foreign-Derived Intangible Income deduction and is dependent on the amount of our U.S. taxable income.     We recorded a provision for income taxes in 2023 as compared to an income tax benefit in 2022 primarily due to an increase in p

## 6: Compare Generated Responses With & Without Reranking

In [7]:
# Generate response without reranking
print("WITHOUT RERANKING:")

# Format the prompt
prompt_without_reranking = format_llama3_prompt(query, context_without_reranking)

# Generate the response
response_without_reranking = enhanced_generate_sagemaker_response(
    prompt=prompt_without_reranking,
    endpoint_name=sagemaker_endpoint,
    generation_config=generation_configuration
)

# Print the query and answer
print("Question:", query)
print("Answer:", response_without_reranking)

WITHOUT RERANKING:
Question: Compare the results between 2022 and 2023
Answer: 

Based on the provided text, here are the key differences between 2022 and 2023:

**Net Income:**

* 2022: Net income of $30,425 million
* 2023: Net income of $59,248 million (an increase of $28,823 million or 95%)

**Operating Income:**

* 2022: Operating income of $12,248 million
* 2023: Operating income of $36,852 million (an increase of $24,604 million or 200%)

**Sales:**

* 2022: North America sales increased 13%
* 2023: North America sales increased 12%
* 2022: International sales decreased 8%
* 2023: International sales increased 11%
* 2022: AWS sales increased 29%
* 2023: AWS sales increased 13%

**Tax Benefits:**

* 2022: The company recorded an income tax benefit
* 2023: The company recorded a provision for income taxes

**Cash Flow:**

* 2022: Cash provided by (used in) financing activities was $9.7 billion
* 2023: Cash used in financing activities was $(15.9) billion

**Other:**

* 2022: The co

In [8]:
# Generate response with reranking
print("WITH RERANKING:")

# Format the prompt
prompt_with_reranking = format_llama3_prompt(query, context_with_reranking)

# Generate the response
response_with_reranking = enhanced_generate_sagemaker_response(
    prompt=prompt_with_reranking,
    endpoint_name=sagemaker_endpoint,
    generation_config=generation_configuration
)

# Print the query and answer
print("Question:", query)
print("Answer:", response_with_reranking)

WITH RERANKING:
Question: Compare the results between 2022 and 2023
Answer: 

Based on the provided text, here are the key results comparisons between 2022 and 2023:

**Operating Income (Loss)**

* 2022: Operating loss of $12.2 billion
* 2023: Operating income of $36.9 billion (a significant increase of $49.1 billion)

**North America**

* 2022: Operating loss of $2.8 billion
* 2023: Operating income of $14.9 billion (a significant increase of $17.7 billion)

**International**

* 2022: Operating loss of $7.7 billion
* 2023: Operating loss of $2.7 billion (a decrease of $5.0 billion)

**AWS**

* 2022: Operating income of $22.8 billion
* 2023: Operating income of $24.6 billion (an increase of $1.8 billion)

**Net Sales**

* 2022: Not specified
* 2023: Expected to be between $121.0 billion and $126.0 billion (an increase of 4% to 8% compared to 2022)

**Net Income**

* 2022: Not specified
* 2023: Net income of $59.2 billion (a significant increase from 2022)

**Cash Provided by (Used in) 

## 7: All-in-One Comparison (Alternative Approach)

In [9]:
# Using the comprehensive function to compare results
query = "What are the key financial metrics for Amazon in 2023?"

# Run the comprehensive comparison
comparison_results = compare_reranking(
    query=query,
    knowledge_base_id=kb_id,
    sagemaker_endpoint=sagemaker_endpoint,
    rerank_model_arn=rerank_model_arn,
    generation_config=generation_configuration,
    bedrock_client=bedrock_client,
    num_results=number_of_results,
    region_name=variables["regionName"]
)

# Display comparison of final responses
print("\n-------- FINAL RESPONSE COMPARISON --------\n")
print("QUESTION: ", query)

print("\nRESPONSE WITHOUT RERANKING:")
print(comparison_results["without_reranking"]["response"])

print("\nRESPONSE WITH RERANKING:")
print(comparison_results["with_reranking"]["response"])

WITHOUT RERANKING:

TOP 3 DOCUMENTS WITHOUT RERANKING:
Position 1 (Score: 0):
These forward-looking statements reflect     Amazon.com’s expectations as of February 2, 2023, and are subject to substantial uncertainty. Our results are inherently unpredictable and may be materially affected by many factors, such as uncertainty regarding the impacts of the COVID-19 pandemic, fluc...

Position 2 (Score: 0):
These forward-looking statements reflect Amazon.com’s expectations as of February 1, 2024, and are subject to substantial uncertainty. Our results are inherently unpredictable and may be materially affected by many factors, such as fluctuations in foreign exchange rates, changes in global economic a...

Position 3 (Score: 0):
These forward-looking statements reflect     Amazon.com’s expectations as of February 6, 2025, and are subject to substantial uncertainty. Our results are inherently unpredictable and may be materially affected by many factors, such as fluctuations in foreign exchan